In [1]:
import librosa
import numpy as np

def preprocess_audio(file_path, sr=22050, duration=5):
    try:
        # Cargar el audio
        audio, _ = librosa.load(file_path, sr=sr)
        # Calcular la longitud deseada en muestras
        target_length = sr * duration
        # Ajustar la longitud del audio: cortar o rellenar silencio
        audio = librosa.util.fix_length(audio, size=target_length)
        return audio
    except Exception as e:
        print(f"Error al procesar el archivo {file_path}: {e}")
        return None

In [2]:
def extract_features(audio, sr=22050, n_mfcc=13):
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)  

In [3]:
import os

def load_dataset(data_dir, sr=22050, duration=5, n_mfcc=13):
    X, y = [], []
    classes = os.listdir(data_dir)  # Cada carpeta es una clase
    for label, cls in enumerate(classes):
        class_path = os.path.join(data_dir, cls)
        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            try:
                audio = preprocess_audio(file_path, sr, duration)
                features = extract_features(audio, sr, n_mfcc)
                X.append(features)
                y.append(label)
            except Exception as e:
                print(f"Error con el archivo {file_path}: {e}")
    return np.array(X), np.array(y), classes


In [4]:
from sklearn.model_selection import train_test_split

X, y, classes = load_dataset("C:\\Users\\otice\\Downloads\\Petmood\\datasetmod\\dogbark")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),  
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(classes), activation='softmax')
])

In [6]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [7]:
model.fit(
    X_train, 
    y_train, 
    epochs=50, 
    batch_size=16, 
    validation_split=0.1
)

Epoch 1/50


6/6 [==============================] - 3s 102ms/step - loss: 8.5983 - accuracy: 0.3000 - val_loss: 10.1657 - val_accuracy: 0.2727
Epoch 2/50
6/6 [==============================] - 0s 36ms/step - loss: 6.7460 - accuracy: 0.3444 - val_loss: 1.0672 - val_accuracy: 0.6364
Epoch 3/50
6/6 [==============================] - 0s 19ms/step - loss: 5.1102 - accuracy: 0.4444 - val_loss: 0.9770 - val_accuracy: 0.2727
Epoch 4/50
6/6 [==============================] - 0s 19ms/step - loss: 4.4023 - accuracy: 0.3556 - val_loss: 3.0525 - val_accuracy: 0.1818
Epoch 5/50
6/6 [==============================] - 0s 19ms/step - loss: 3.5711 - accuracy: 0.2889 - val_loss: 1.6470 - val_accuracy: 0.2727
Epoch 6/50
6/6 [==============================] - 0s 20ms/step - loss: 3.4078 - accuracy: 0.3444 - val_loss: 1.0527 - val_accuracy: 0.4545
Epoch 7/50
6/6 [==============================] - 0s 23ms/step - loss: 2.3187 - accuracy: 0.4111 - val_loss: 0.9080 - val_accuracy: 0.4545
Epoch 8/50
6/6 [=======

In [8]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

1/1 [==============================] - 0s 86ms/step - loss: 0.8428 - accuracy: 0.7500
Validation accuracy: 75.00%


In [9]:
model.save("modelo_bark.h5")

c:\Users\otice\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
from keras.models import load_model
model = load_model("modelo_bark.h5")

In [11]:
def predict_audio(file_path, model, classes, sr=22050, duration=5, n_mfcc=13):
    audio = preprocess_audio(file_path, sr, duration)
    features = extract_features(audio, sr, n_mfcc)
    features = np.expand_dims(features, axis=0)  
    prediction = model.predict(features)
    predicted_class = classes[np.argmax(prediction)]
    return predicted_class

archivo = "C:\\Users\\otice\\Downloads\\Petmood\\datasetmod\\dogbark\\Alerta o emoción\\dog_10.wav"
print("Predicción:", predict_audio(archivo, model, classes))

1/1 [==============================] - 0s 258ms/step
Predicción: Advertencia o miedo


In [ ]:
#Licenciamiento de cada una de las herramientas utilizadas o frameworks